# Features

## Question:We want to highlight 10 wines to increase our sales. Which ones should we choose and why? 

In [1]:
import sqlite3
connexion = sqlite3.connect("../db/vivino.db")
cursor = connexion.cursor()

In [2]:
def top_wines(score: float):    
    try:
        
        cursor.execute("""
        CREATE VIEW top10_wines AS
        SELECT w.name, ROUND((w.ratings_average + v.ratings_average), 3) AS total_score
        FROM wines AS w
        JOIN vintages AS v ON w.id = v.wine_id
        JOIN wineries AS wi ON w.winery_id = wi.id
        ORDER BY v.ratings_average DESC
        LIMIT 10
    """)
        connexion.commit()

        print("view created")
    except Exception as e:
        print("Error:", e)
        connexion.rollback()  # Rollback changes if an error occurs

Error: view top10_wines already exists


In [3]:

cursor.execute("""
    SELECT *
    from top10_wines                   

""")
cursor.fetchall()

[("L'Extravagant de Doisy-Daëne Sauternes", 9.5),
 ('Brunello di Montalcino Riserva', 9.5),
 ('Pauillac (Premier Grand Cru Classé)', 9.4),
 ('Clos du Mesnil', 9.5),
 ('Le Mesnil Blanc de Blancs (Cuvée S) Brut Champagne', 9.5),
 ('Amarone della Valpolicella Monte Lodoletta', 9.5),
 ('Harlan Estate Red', 9.5),
 ('Clos du Mesnil', 9.5),
 ('Vintage Port Nacional', 9.4),
 ('Pomerol', 9.5)]

## Question2: We detected that a big cluster of customers likes a specific combination of tastes. We identified a few keywords that match these tastes: coffee, toast, green apple, cream, and citrus (note that these keywords are case sensitive ⚠️). We would like you to find all the wines that are related to these keywords. Check that at least 10 users confirm those keywords, to ensure the accuracy of the selection. Additionally, identify an appropriate group name for this cluster.

In [43]:
try:
    cursor.execute("""
    CREATE VIEW top_keywords AS
    SELECT w.name AS wine_name,
        GROUP_CONCAT(DISTINCT k.name) AS keywords
    FROM wines AS w
    JOIN keywords_wine AS kw ON w.id = kw.wine_id
    JOIN keywords AS k ON kw.keyword_id = k.id
    WHERE k.name IN ('coffee', 'toast', 'green apple', 'cream', 'citrus')
    GROUP BY w.name
    HAVING MIN(kw.count) >= 10;
    """)
    connexion.commit()

    print("view created")
except Exception as e:
    print("Error:", e)
    connexion.rollback()  # Rollback changes if an error occurs

view created


In [44]:

cursor.execute("""
    SELECT *
    from top_keywords                   

""")
cursor.fetchall()

[('Almaviva', 'coffee,toast,cream'),
 ("Avizoise Blanc de Blancs Extra Brut Champagne Grand Cru 'Avize'",
  'cream,toast,green apple,citrus'),
 ('Belle Epoque Brut Champagne', 'green apple,toast,coffee,citrus,cream'),
 ('Belondrade y Lurton', 'green apple,cream,toast,citrus'),
 ('Blanc des Millénaires', 'toast,coffee,cream,citrus,green apple'),
 ("Bricco dell'Uccellone Barbera d'Asti", 'coffee,toast,cream'),
 ('Brut Champagne', 'toast,coffee,cream,green apple,citrus'),
 ('CASK 23 Cabernet Sauvignon', 'coffee,toast,cream'),
 ('Campolongo di Torbe Amarone della Valpolicella Classico', 'coffee'),
 ('Chassagne-Montrachet', 'cream,toast,citrus,green apple'),
 ('Comtes de Champagne Blanc de Blancs',
  'toast,coffee,cream,citrus,green apple'),
 ('Corton-Charlemagne Grand Cru Blanc', 'cream,toast,citrus,green apple'),
 ('Cristal Brut Champagne (Millésimé)',
  'toast,coffee,green apple,citrus,cream'),
 ('Cuvée des Enchanteleurs Brut Champagne',
  'toast,coffee,cream,green apple,citrus'),
 ('Dom

## Question7: One of our VIP clients likes Cabernet Sauvignon and would like our top 5 recommendations. Which wines would you recommend to him?

In [20]:
try:
    cursor.execute("""
    DROP VIEW IF EXISTS wines_Cabernet;
    """)

    cursor.execute("""
    create view wines_Cabernet AS
    SELECT name, ratings_average, ratings_count
    FROM wines
    WHERE LOWER(name) LIKE "%cabernet sauvignon%"
    ORDER BY ratings_average DESC, ratings_count DESC
    limit 5;
    """)
    connexion.commit()

    print("view created")
except Exception as e:
    print("Error:", e)
    connexion.rollback() 


view created


In [21]:
cursor.execute("""
select *
from wines_cabernet


""")
cursor.fetchall()

[('Cabernet Sauvignon', 4.8, 2941),
 ('Special Selection Cabernet Sauvignon', 4.7, 41236),
 ('Cabernet Sauvignon', 4.6, 157944),
 ('Cabernet Sauvignon (Signature)', 4.6, 13730),
 ('CASK 23 Cabernet Sauvignon', 4.6, 9464)]

## streamlit code

In [13]:
def top_wines(score: float):
    cursor.execute("""
        SELECT w.name, ROUND((w.ratings_average + v.ratings_average), 3) AS total_score
        FROM wines AS w
        JOIN vintages AS v ON w.id = v.wine_id
        JOIN wineries AS wi ON w.winery_id = wi.id
        WHERE ROUND((w.ratings_average + v.ratings_average), 3) >= ?
        ORDER BY ROUND((w.ratings_average + v.ratings_average), 3) ASC
    """, (score,))
    result = cursor.fetchall()
    for row in result:
        print(row)


In [14]:
top_wines(8.5)

('Red', 8.5)
('Capellanía Reserva', 8.5)
('Clos des Porrets Nuits-Saint-Georges 1er Cru', 8.5)
("Montepulciano d'Abruzzo", 8.6)
('Pomerol', 8.6)
("Montepulciano d'Abruzzo", 8.6)
('Amarone Della Valpolicella Marta Galli', 8.6)
('Cuvée Kermesse Rosso', 8.6)
('Barolo Cannubi', 8.6)
('Rioja 150 Aniversario', 8.6)
('La Chapelle de La Mission Haut-Brion Pessac-Léognan', 8.6)
('Cresasso Corvina Veronese', 8.6)
('Kevin Arnold Shiraz', 8.6)
('Margaux (Grand Cru Classé)', 8.6)
('Châteauneuf-du-Pape Chaupin', 8.6)
('Bolgheri Superiore Sondraia', 8.6)
('Barolo Cerretta', 8.6)
('Chafandin', 8.6)
("Aile d'Argent Blanc Bordeaux", 8.6)
('Toscana', 8.6)
('Nuits-Saint-Georges 1er Cru Les Damodes', 8.6)
('Malbec', 8.6)
('Valdegines', 8.6)
('Linticlarus Lagrein Riserva', 8.6)
('Carabantes Syrah', 8.6)
('Châteauneuf-du-Pape', 8.6)
('Old Vines Malbec', 8.6)
('Vieilles Vignes Meursault', 8.6)
('25 anni Montefalco Sagrantino', 8.7)
('Brunello di Montalcino', 8.7)
('Cepparello Red', 8.7)
('Saint-Julien (Grand 

In [17]:
def wines_keynotes(k1, **kwargs):
    # Combine k1 and additional keyword arguments
    keywords = [k1, *kwargs.values()]

    # Filter out None values and convert to strings
    keywords = [str(k) for k in keywords if k is not None]

    # Create the SQL IN clause string with the keywords
    in_clause = ",".join(f"'{k}'" for k in keywords)

    # Construct the SQL query dynamically
    query = f"""
    SELECT w.name AS wine_name,
        GROUP_CONCAT(DISTINCT k.name) AS keywords
    FROM wines AS w
    JOIN keywords_wine AS kw ON w.id = kw.wine_id
    JOIN keywords AS k ON kw.keyword_id = k.id
    WHERE k.name IN ({in_clause})
    GROUP BY w.name
    HAVING MIN(kw.count) >= 10;
    """

    # Execute the query
    cursor.execute(query)
    result = cursor.fetchall()

    # Return the result
    return result



In [20]:
wines_keynotes(k1='toast', k2='citrus', k3='green apple', k4='cream')


[("'Giulio Ferrari' Riserva del Fondatore", 'cream,toast,green apple,citrus'),
 ('5 Puttonyos Aszú (Blue Label)', 'green apple,citrus,toast,cream'),
 ("Aile d'Argent Blanc Bordeaux", 'green apple,citrus,cream,toast'),
 ('Almaviva', 'toast,cream'),
 ('Amour de Deutz Millésimé Brut Champagne', 'toast,cream,citrus,green apple'),
 ("Avizoise Blanc de Blancs Extra Brut Champagne Grand Cru 'Avize'",
  'cream,toast,green apple,citrus'),
 ('Belle Epoque Brut Champagne', 'green apple,toast,citrus,cream'),
 ('Belondrade y Lurton', 'green apple,cream,toast,citrus'),
 ('Blanc de Blancs Brut Champagne (Vintage)',
  'green apple,citrus,toast,cream'),
 ('Blanc de Blancs Brut Champagne Grand Cru',
  'toast,cream,green apple,citrus'),
 ('Blanc des Millénaires', 'toast,cream,citrus,green apple'),
 ("Bricco dell'Uccellone Barbera d'Asti", 'toast,cream'),
 ('Brut Champagne', 'toast,cream,green apple,citrus'),
 ('Brut Champagne (Gold)', 'green apple,toast,citrus,cream'),
 ('Brut Cuvée Champagne Rosé', 'cre